## Make GPT vs. Expert Model Human Annotation Data

In [1]:
import json
import rich
import random

In [2]:
with open("/home/wenkail/iclr_rebuttal/llm_personality/llm_bigfive/data_construction_baseline/classifier_on_posts/gpt4omini/out_gpt4omini.jsonl", "r") as f:
    data = []
    for i in f.readlines():
        data.append(json.loads(i))

traits = ["o", "c", "e", "a", "n"]
predict_dict = {}
for t in traits:
    with open(f"/data/user_data/wenkail/llm_personality/generator/data/outputs/generated_predictions_{t}.jsonl", "r") as f:
        store = []
        for i in f.readlines():
            store.append(json.loads(i))
        predict_dict[t] = store
    
    
with open("/home/wenkail/iclr_rebuttal/llm_personality/llm_bigfive/data_construction_baseline/classifier_on_posts/data_input.json", "r") as f:
    data_input = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/data/user_data/wenkail/llm_personality/generator/data/outputs/generated_predictions_o.jsonl'

In [77]:
sum = 0
trait = ["o", "c","e", "a","n"]
for t in trait:
    for l in ["low","high"]:
        sum += len([d for d in data_input if d['trait'] ==t and d['level']==l])
        print(f"{t} {l} {len([d for d in data_input if d['trait'] == t and d['level'] == l])}")

o low 105
o high 95
c low 99
c high 101
e low 107
e high 93
a low 97
a high 103
n low 101
n high 99


In [78]:
for i in range(len(data_input)):
    data[i]['trait'] = data_input[i]['trait']
    data[i]['level'] = data_input[i]['level']

In [79]:
predict_dict['o'][0]

{'prompt': 'user\n\nHelp me complete the sentence with certain Big Five Personality: Openness - low\ni demand that ross playassistant\n\n',
 'label': "that game until he is intel's mindslave. shouldn't take long.",
 'predict': 'the game'}

In [ ]:
store_predict = {}
# init
for t in trait:
    store_predict[t] = {}
    for l in ["low", "high"]:
        store_predict[t][l] = []
        
for t in trait:
    for i in range(len(predict_dict[t])):
        try:
            level = predict_dict[t][i]['prompt'].split("\n")[-4].split(":")[-1].strip().split("-")[1].strip()
            store_predict[t][level].append(predict_dict[t][i]['prompt'].split('\n')[-3] + " " + predict_dict[t][i]['predict'])
        except:
            print(f"Error happened at {t} {i}")
            continue
        # predict_dict[i]['prompt'].pop(key)

Error happened at a 108


In [81]:
len(store_predict['o']['high'])

489

In [82]:
import random
sub_predict = {}
# init
for t in trait:
    sub_predict[t] = {}
    for l in ["low", "high"]:
        sub_predict[t][l] = []
        
for t in traits:
    for l in ["low", "high"]:
        sample_size = min(len(store_predict[t][l]), len([d for d in data_input if d['trait'] == t and d['level'] == l]))
        random_sample = random.sample(store_predict[t][l], sample_size)
        sub_predict[t][l] = random_sample
# sub_predict['o']['high']
        

In [87]:
for d in data:
    d['gpt-4o-mini'] = d['post']
    d['expert'] = sub_predict[d['trait']][d['level']].pop()
    d.pop('post')
    d.pop('prompt2')
    d.pop('topic')
    d.pop('prompt1')
data

[{'trait': 'o',
  'level': 'high',
  'gpt-4o-mini': '- sometimes, in the midst of all this beautiful chaos, you stumble upon those perfect little moments that make it all worthwhile. like when the sun sets just right and the world feels like a painting. it’s like the universe is whispering, “hey, you’re exactly where you need to be.” ✨🎨 #FindingPerfection #ArtOfLife',
  'expert': "been a little ill thisassistant weekend, but i'm feeling better now."},
 {'trait': 'o',
  'level': 'low',
  'gpt-4o-mini': "so there's this health day event coming up about sexual education. not really my thing, but i guess they’re giving away some funny stuff related to sperm? like, who even thinks of that? anyway, i’ll probably just stick to my usual routine and avoid the crowd. but hey, if you’re into that sort of thing, good luck!",
  'expert': "i'm at daniela's house savingassistant her from her mom."},
 {'trait': 'o',
  'level': 'low',
  'gpt-4o-mini': "i don't really get the whole bubble gum game thing

In [88]:
import pandas as pd
df = pd.DataFrame(data)
df

,trait,level,gpt-4o-mini,expert
0,o,high,"- sometimes, in the midst of all this beautifu...","been a little ill thisassistant weekend, but i..."
1,o,low,so there's this health day event coming up abo...,i'm at daniela's house savingassistant her fro...
2,o,low,i don't really get the whole bubble gum game t...,well i guess this willassistant be my last pos...
3,o,high,"- mornings have been a real struggle lately, l...",i'm at the doctors... haven'tassistant been he...
4,o,low,just got some yarn made from possum and merino...,i never asked my belovedassistant to be a man.
...,...,...,...,...
995,n,low,- it's been amazing to see our little forum gr...,about to take a fatassistant ass nap
996,n,low,"- some days, the best adventures happen right ...","boy, george...i just finished starassistant wa..."
997,n,low,- you ever notice how cardinals are like the o...,wow!i suppose i got throughassistant the first...
998,n,low,"- you know, sometimes words can feel a bit lik...",me and desi rolling downassistant the street


In [ ]:
# df.to_csv("/home/wenkail/iclr_rebuttal/llm_personality/llm_bigfive/data_construction_baseline/classifier_on_posts/human_eval_gpt_4o_mini_vs_expert.csv")

## Make Human Annotation Data: Big5-Chat

In [17]:
import json
import pandas as pd
import random

In [2]:
with open("/data/user_data/wenkail/llm_personality/align/data_sft/test.json", "r") as f:
    data = json.load(f)

In [6]:
# print(data[0]['instruction'].split(":")[-1].split("-")[-1].strip())
data[0]
# data[0]['instruction'].split(":")[-1].split("-")[0].strip()

{'instruction': 'You are a helpful assistant with the following Big Five personality traits: Openness - low',
 'input': "Kaelin, I'm so stoked we're on the same page! I mean, think about it - if we can just dive in and start doing, we can figure anything out. No more getting bogged down in theory or worrying about not knowing enough. Let's just go for it and learn as we go!",
 'output': "I don't know, Cherelle. That sounds pretty reckless to me. What if we make mistakes or get in over our heads? I think it's better to take things slow and make sure we're doing it right. We can't just rush into things without a plan."}

In [7]:
anno = []
for d in data:
    store = {}
    store['trait'] = d['instruction'].split(":")[-1].split("-")[0].strip()
    store['level'] = d['instruction'].split(":")[-1].split("-")[-1].strip()
    store['big5_chat'] = d['output']
    anno.append(store)
len(anno)

10000

In [15]:
anno[0]

{'trait': 'Openness',
 'level': 'low',
 'big5_chat': "I don't know, Cherelle. That sounds pretty reckless to me. What if we make mistakes or get in over our heads? I think it's better to take things slow and make sure we're doing it right. We can't just rush into things without a plan."}

In [18]:
anno_dict = {}
trait = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']
level = ['low', 'high']
# init
for t in trait:
    anno_dict[t] = {}
    for l in level:
        anno_dict[t][l] = [] 
        
for a in anno:
    anno_dict[a['trait']][a['level']].append(a['big5_chat'])

# init sub
sub_annos = {}
for t in trait:
    for l in level:
        sub_annos[t] = {}
        sub_annos[t][l] = [] 

# Store the annotation
for t in trait:
    for l in level:
        sample_size = min(len(anno_dict[t][l]), 50)
        random_sample = random.sample(anno_dict[t][l], sample_size)
        sub_annos[t][l] = random_sample

In [ ]:
anno_df = pd.DataFrame(anno)
# anno_df.to_csv("/home/wenkail/iclr_rebuttal/llm_personality/llm_bigfive/data_construction_baseline/classifier_on_posts/human_eval_big5_chat.csv")

In [20]:
len(sub_annos['Openness']['high'])

50

In [22]:
sub_annos_df = pd.DataFrame(anno)
sub_annos_df.to_csv("/home/wenkail/iclr_rebuttal/llm_personality/llm_bigfive/data_construction_baseline/classifier_on_posts/human_eval_big5_chat.csv")